In [82]:
import torch
from PIL import Image
from torchvision.transforms import v2
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from skimage.filters import butterworth
from skimage.filters import gaussian
from skimage import exposure

In [83]:
googleNet = torch.load('Final_Models\googlenet_mura_Norma_p2LionOPT_40epoch.pth')
googleNet = googleNet.to('cpu')

In [84]:
swin = torch.load('Final_Models\SWIN_PART3mura_LION_Normalization_Balanced20epoch.pth')
swin = swin.to('cpu')

# طريقة معالجة الصورة
# Data Flow

In [85]:
image = Image.open(r"MURA-v1.1\valid\XR_WRIST\patient11313\study1_negative\image2.png")
image = image.convert('RGB')
 # Convert PIL Image to numpy array
image_np = np.array(image)

      
image_gray = rgb2gray(image_np)        
        
        
image_gray = exposure.equalize_adapthist(image_gray, clip_limit=0.02)

        
image_butterworth = butterworth(image_gray)


image_gaussian = gaussian(image_gray)
        
# Combine the two images into one (stack them along the last axis)
# Create a 3 channel image from the butterworth and gaussian filtered images
image_combined = np.stack([image_butterworth, image_gaussian, image_gray], axis=-1)
        
image_combined = (image_combined - np.min(image_combined)) / (np.max(image_combined) - np.min(image_combined))
        
        # Convert to 8-bit image for PIL
image_combined = img_as_ubyte(image_combined)
        
        # Convert numpy array back to PIL Image
image = Image.fromarray(image_combined)

mean_values = [0.2665, 0.4648, 0.4648]# Calculated
std_values = [0.1079, 0.1650, 0.1666] # Calculated
val_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    v2.Normalize(mean=mean_values, std=std_values)

])
image = val_transform(image)
transformed_image = image.unsqueeze(0)

# احتمال صورة واحدة

In [86]:
with torch.no_grad():
    googleNet_output = googleNet(transformed_image)
    googleNet_proba = F.softmax(googleNet_output, dim=1)
    
    # _, predicted_indices = torch.max(googleNet_output, 1)
    # predicted_index = predicted_indices.item()  # Convert tensor to Python number
    

In [87]:
print(googleNet_proba)

tensor([[0.9944, 0.0056]])


In [88]:
with torch.no_grad():
    swin_output = swin(transformed_image)
    logits = swin_output.logits 
    swin_proba = F.softmax(logits, dim=1)

    _, predicted_indices = torch.max(logits, 1)
    predicted_index = predicted_indices.item()  # Convert tensor to Python number

In [89]:
print(logits)

tensor([[ 3.8530, -4.3215]])


In [90]:
swin_proba_np = swin_proba.numpy()
googleNet_proba_np = googleNet_proba.numpy()


In [91]:
probabilities = np.array([swin_proba_np, googleNet_proba_np])
print(probabilities)


[[[9.9971837e-01 2.8166702e-04]]

 [[9.9440902e-01 5.5909292e-03]]]


In [92]:
average_probabilities = np.mean(probabilities, axis=0)
ensemble_prediction = np.argmax(average_probabilities)  

In [93]:
average_probabilities

array([[0.9970637, 0.0029363]], dtype=float32)

In [94]:
ensemble_prediction

0

# النتيجة

In [96]:
if ensemble_prediction == 0:
        print('Normal')
elif ensemble_prediction == 1:
        print('Abnormal')

Normal
